In [1]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import math
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

In [2]:
#to create custom dataset pytorch inbuilt class used

from torch.utils.data import Dataset, DataLoader

class TextDataset(Dataset):

    def __init__(self, english_sentences, lang_sentences):
        self.english_sentences = english_sentences
        self.lang_sentences = lang_sentences
    #rturn number of eng/hind or sentences in a list
    def __len__(self):
        return len(self.english_sentences)
    #return 1:1 mapping of one lang to other
    def __getitem__(self, idx):
        return self.english_sentences[idx], self.lang_sentences[idx]



In [3]:

class SentenceEmbedding(nn.Module):
    "For a given sentence, create an embedding"
    def __init__(self, max_sequence_length, d_model, language_to_index, START_TOKEN, END_TOKEN, PADDING_TOKEN):
        super().__init__()
        self.vocab_size = len(language_to_index)
        self.max_sequence_length = max_sequence_length
        self.embedding = nn.Embedding(self.vocab_size, d_model,padding_idx=0)
        self.language_to_index = language_to_index
        self.position_encoder = RoPEEmbedding(d_model)  # Ensure RoPEEmbedding is defined
        self.dropout = nn.Dropout(p=0.1)
        self.START_TOKEN = START_TOKEN
        self.END_TOKEN = END_TOKEN
        self.PADDING_TOKEN = PADDING_TOKEN


    def batch_tokenize(self, batch, start=False, end=False):
        def tokenize(sentence):
            # Convert each character/token to its index
            sentence_word_indices = [self.language_to_index.get(token, self.language_to_index[self.PADDING_TOKEN]) for token in list(sentence)]

            if start:
                sentence_word_indices.insert(0, self.language_to_index[self.START_TOKEN])
            if end:
                sentence_word_indices.append(self.language_to_index[self.END_TOKEN])

            # Ensure all sequences are exactly max_sequence_length long
            sentence_word_indices = sentence_word_indices[:self.max_sequence_length]  # Truncate if too long
            sentence_word_indices += [self.language_to_index[self.PADDING_TOKEN]] * (self.max_sequence_length - len(sentence_word_indices))  # Pad if too short

            return torch.tensor(sentence_word_indices)

        tokenized = [tokenize(sentence) for sentence in batch]
        tokenized = torch.stack(tokenized)  # Convert list of tensors to a single tensor

        return tokenized  #(batch,seqlen)

    def forward(self, x, start=False, end=False):
        x = self.batch_tokenize(x, start, end)  # Ensure consistent function calls
        x = self.embedding(x)  # Convert token indices to embeddings
        #pos = self.position_encoder(x)  # Apply positional encoding
        #x = self.dropout(x + pos)  # Apply dropout
        return x #(batch,seqlen)


#rope embedding
class RoPEEmbedding(torch.nn.Module):
    def __init__(self, embedding_dim):
        super().__init__()
        assert embedding_dim % 2 == 0, "Embedding dimension must be even for RoPE"
        self.embedding_dim = embedding_dim

    def forward(self, x):
        """
        Forward pass for Rotary Position Embedding.

        Args:
        - x: Tensor of shape (batch_size, seq_len, embedding_dim)

        Returns:
        - Tensor with RoPE applied to the last two dimensions.
        """
        seq_len = x.shape[1]

        # Generate position indices
        position_ids = torch.arange(seq_len, dtype=torch.float32, device=x.device)

        # Compute the rotary angles
        freqs = 1.0 / (10000 ** (torch.arange(0, self.embedding_dim, 2, dtype=torch.float32, device=x.device) / self.embedding_dim))
        angles = torch.einsum('i,j->ij', position_ids, freqs)

        # Create the rotation matrix for sin and cos embeddings
        sin = torch.sin(angles).repeat_interleave(2, dim=-1)
        cos = torch.cos(angles).repeat_interleave(2, dim=-1)

        # Apply rotation using cos and sin embeddings
        x1 = x * cos + self.rotate_half(x) * sin
        return x1


    def rotate_half(self,x):
        """
        Rotate the last dimension of the input tensor by swapping odd and even elements and negating one.

        Args:
        - x: Tensor of shape (..., embedding_dim)

        Returns:
        - Rotated tensor of the same shape.
        """
        x1, x2 = x[..., ::2], x[..., 1::2]  # Split into even and odd dimensions
        return torch.cat((-x2, x1), dim=-1)

#sine embedding
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, max_sequence_length):
        super().__init__()
        self.max_sequence_length = max_sequence_length
        self.d_model = d_model

    def forward(self, x):
        batch_size, seq_len, _ = x.size()

        # Generate the positional encoding
        even_i = torch.arange(0, self.d_model, 2).float()
        denominator = torch.pow(10000, even_i/self.d_model)
        position = torch.arange(self.max_sequence_length).reshape(self.max_sequence_length, 1)
        even_PE = torch.sin(position / denominator)
        odd_PE = torch.cos(position / denominator)
        stacked = torch.stack([even_PE, odd_PE], dim=2)
        PE = torch.flatten(stacked, start_dim=1, end_dim=2)

        # Ensure the PE matches the batch size and sequence length
        PE = PE.unsqueeze(0).expand(batch_size, seq_len, self.d_model)

        return PE + x

In [4]:

#feedforward network
class feedforward(nn.Module):
  def __init__(self,d_model,hidlayer,dropout):
    super().__init__()
    self.d_model=d_model
    self.hidlayer=hidlayer
    self.linearlayer1=nn.Linear(self.d_model,self.hidlayer)
    self.linearlayer2=nn.Linear(self.hidlayer,self.d_model)
    self.dropout=nn.Dropout(dropout)
    self.activation=nn.ReLU()

  def forward(self,x):
    l1=self.linearlayer1(x)
    l1=self.activation(l1)
    l1=self.dropout(l1)
    out=self.linearlayer2(l1)
    out=self.dropout(out)
    return out

#multhead attention
class multihead_attention(nn.Module):
    def __init__(self,dmodel,heads=1):
        super().__init__()
        self.heads = heads
        self.dmodel=dmodel
        assert self.dmodel % self.heads == 0, "Embedding dimension must be divisible by num_heads"

        self.dmodel = dmodel
        self.head_dim = self.dmodel // self.heads
        self.wq = nn.Linear(self.dmodel, self.dmodel)
        self.wk = nn.Linear(self.dmodel, self.dmodel)
        self.wv = nn.Linear(self.dmodel, self.dmodel)
        self.linearlayer=nn.Linear(self.dmodel,self.dmodel)



    def scaled_dot_product_attention(self, q, k, v,padding_mask):
        dk = torch.tensor(q.shape[-1], dtype=torch.float32)
        scaled = torch.matmul(q, k.transpose(-2, -1)) / torch.sqrt(dk)

        if padding_mask is not None:
            scaled = scaled.masked_fill(padding_mask==1,-1e9)


        attention = torch.softmax(scaled, dim=-1)
        scores = torch.matmul(attention, v)
        return attention, scores

    def forward(self, x,padding_mask):
        batch_size, sequence_length, input_dim = x.size()
        q = self.wq(x)
        k = self.wk(x)
        v = self.wv(x)
        q = q.view(batch_size, sequence_length, self.heads, self.head_dim)
        k = k.view(batch_size, sequence_length, self.heads, self.head_dim)
        v = v.view(batch_size, sequence_length, self.heads, self.head_dim)
        q = q.permute(0, 2, 1, 3)
        k = k.permute(0, 2, 1, 3)
        v = v.permute(0, 2, 1, 3)

        attention, scores = self.scaled_dot_product_attention(q, k, v,padding_mask)
        scores = scores.reshape(batch_size, sequence_length, self.heads *self.head_dim)# we can use self.dmodel as well as last arg
        out=self.linearlayer(scores)
        return out



#layer norm
class CustomLayerNorm(nn.Module):
    def __init__(self, normalized_shape, epsilon=1e-5):
        super(CustomLayerNorm, self).__init__()
        self.gamma = nn.Parameter(torch.ones(normalized_shape))
        self.beta = nn.Parameter(torch.zeros(normalized_shape))
        self.epsilon = epsilon

    def forward(self, x):
        # Calculate mean and std across the last dimension (features) for each sequence in the batch
        mean = x.mean(dim=-1, keepdim=True)
        std = x.std(dim=-1, keepdim=True)
        x_normalized = (x - mean) / (std + self.epsilon)

        # Apply gamma and beta, which are learned parameters for normalization
        # The shape of gamma and beta should match the feature size
        return self.gamma.unsqueeze(0).unsqueeze(0) * x_normalized + self.beta.unsqueeze(0).unsqueeze(0)



class multihead_cross_attention(nn.Module):
    def __init__(self, dmodel, heads=1):
        super().__init__()
        self.heads = heads

        assert dmodel % heads == 0, "Embedding dimension must be divisible by num_heads"

        self.dmodel = dmodel
        self.head_dim = self.dmodel // self.heads
        self.wqc = nn.Linear(self.dmodel, self.dmodel)
        self.wkc = nn.Linear(self.dmodel, self.dmodel)
        self.wvc = nn.Linear(self.dmodel, self.dmodel)
        self.linearlayer=nn.Linear(self.dmodel,self.dmodel)

    def scaled_dot_product_attention(self, q, k, v,look_ahead_mask):
        dk = torch.tensor(q.shape[-1], dtype=torch.float32)
        scaled = torch.matmul(q, k.transpose(-2, -1)) / torch.sqrt(dk)



        if look_ahead_mask is not None:
            scaled = scaled.masked_fill(look_ahead_mask==0,-1e9)

        attention = torch.softmax(scaled, dim=-1)
        scores = torch.matmul(attention, v)
        return attention, scores

    def forward(self, x,y,look_ahead_mask):
        batch_size, sequence_length, input_dim = x.size()
        # if y.size(0) != x.size(0):
        #      pad_size = x.size(0) - y.size(0)
        #      padding = torch.zeros(pad_size, y.size(1), y.size(2), device=y.device)
        #      y = torch.cat([y, padding], dim=0)

        #print('x and y shape cross attention',x.shape,y.shape)

        # q and k from decoder
        q = self.wqc(x)
        k = self.wkc(y)
        # v from decoder
        v = self.wvc(y)
        q = q.view(batch_size, sequence_length, self.heads, self.head_dim)
        k = k.view(batch_size, sequence_length, self.heads, self.head_dim)
        v = v.view(batch_size, sequence_length, self.heads, self.head_dim)
        q = q.permute(0, 2, 1, 3)
        k = k.permute(0, 2, 1, 3)
        v = v.permute(0, 2, 1, 3)

        attention, scores = self.scaled_dot_product_attention(q, k, v,look_ahead_mask)
        scores = scores.reshape(batch_size, sequence_length, self.heads *self.head_dim)# we can use self.dmodel as well as last arg
        out=self.linearlayer(scores)
        return out




In [5]:
#encoder


#encoder layer

class encoderlayer(nn.Module):
  def __init__(self,max_sequence_length,d_model,hidlayer,dropout,num_heads):
    super().__init__()
    self.d_model,self.hidlayer,self.dropout,self.num_heads=d_model,hidlayer,dropout,num_heads
    self.multihead_attention=multihead_attention(d_model,num_heads)
    self.feedforward=feedforward(self.d_model,self.hidlayer,self.dropout)
    self.layernorm1=CustomLayerNorm(self.d_model)
    self.layernorm2=CustomLayerNorm(self.d_model)
    self.max_sequence_length=max_sequence_length

  def forward(self,x,padding_mask):
    #mulihead
    mha=self.multihead_attention(x,padding_mask)
    #layernorm
    ln1=self.layernorm1(mha+x)
    #feedforward
    ff=self.feedforward(ln1)
    #layernorm
    out=self.layernorm2(ff+ln1)
    return out



class encoder(nn.Module):
  #creating n layer of layers
  def __init__(self,max_sequence_length,d_model,hidlayer,dropout,num_heads,nlayers,language_to_index,START_TOKEN,END_TOKEN,PADDING_TOKEN
               ):
    super().__init__()
    #sequentially stack encoders
    self.encoderembeddings=SentenceEmbedding(max_sequence_length,d_model,language_to_index,START_TOKEN,END_TOKEN,PADDING_TOKEN)
    self.layers=nn.Sequential(*[encoderlayer(max_sequence_length,d_model,hidlayer,dropout,num_heads) for _ in range(nlayers)])

  def forward(self, x,padding_mask,start,end):
    x=self.encoderembeddings(x,)
    for i, layer in enumerate(self.layers):
      #print(f'\n------layer {i+1}----- ')

      x = layer(x,padding_mask)
      #print(f"--Output after layer {i+1}--: {x.size()}")  # Printing the size after each layer
    return x




In [6]:
#decoder
class decoderlayer(nn.Module):
  def __init__(self,max_sequence_length,d_model,hidlayer,dropout,num_heads):
    super().__init__()
    self.d_model,self.hidlayer,self.dropout,self.num_heads=d_model,hidlayer,dropout,num_heads
    #self.PositionalEncoding=PositionalEncoding(self.d_model,self.input_dim) #sinencoding
    self.multihead_attention=multihead_attention(self.d_model,self.num_heads)
    self.layernorm1=CustomLayerNorm(self.d_model)
    self.cross_attention=multihead_cross_attention(d_model,num_heads)
    self.layernorm2=CustomLayerNorm(self.d_model)
    self.feedforward=feedforward(self.d_model,self.hidlayer,self.dropout)
    self.layernorm3=CustomLayerNorm(self.d_model)
    self.max_sequence_length=max_sequence_length


  def forward(self,x,y,padding_mask,look_ahead_mask):
    if y.size(0) != x.size(0):
             pad_size = x.size(0) - y.size(0)
             padding = torch.zeros(pad_size, y.size(1), y.size(2), device=y.device)
             y = torch.cat([y, padding], dim=0)

    #multihead attention
    mha=self.multihead_attention(y,padding_mask)
    #layernorm1
    ln1=self.layernorm1(mha+y)
    #cross attention
    cmha=self.cross_attention(x,ln1,look_ahead_mask)
    #layernorm2
    ln2=self.layernorm2(cmha+ln1)

    #feedforward
    ff=self.feedforward(ln1)
    #layernorm3
    out=self.layernorm3(ff+ln2)
    return out

class decoder(nn.Module):
  #creating n layer of layers
  def __init__(self,max_sequence_length,d_model,hidlayer,dropout,num_heads,nlayers,language_to_index,START_TOKEN,END_TOKEN,PADDING_TOKEN
               ):
    super().__init__()
    #sequentially stack encoders
    self.decoderembeddings=SentenceEmbedding(max_sequence_length,d_model,language_to_index,START_TOKEN,END_TOKEN,PADDING_TOKEN)
    self.layers=nn.Sequential(*[decoderlayer(max_sequence_length,d_model,hidlayer,dropout,num_heads) for _ in range(nlayers)])

  def forward(self, x,y,start,end,padding_mask,look_ahead_mask):
    y = self.decoderembeddings(y, start,end)
    for i, layer in enumerate(self.layers):
      #print(f'\n------layer {i+1}----- ')
      l = layer(x,y,padding_mask,look_ahead_mask)
      #print(f"--Output after layer {i+1}--: {l.size()}")  # Printing the size after each layer
    return l




In [7]:
#now the transformer class
#transformer class
class mytransformer(nn.Module):
  def __init__(self,
               d_model,
               hidlayer,
               dropout,
               num_heads,
               nlayers,
               lang_vocab_size,
               english_to_index,
               lang_to_index,
               max_seq_len,
               START_TOKEN,
               END_TOKEN,
               PADDING_TOKEN):

    super().__init__()
    self.d_model=d_model
    self.hidlayer=hidlayer
    self.dropout=dropout
    self.num_heads=num_heads
    self.nlayers=nlayers
    self.vocab_size=lang_vocab_size
    self.english_to_index = english_to_index
    self.lang_to_index = lang_to_index
    self.max_sequence_length = max_seq_len
    self.START_TOKEN = START_TOKEN
    self.END_TOKEN = END_TOKEN
    self.PADDING_TOKEN = PADDING_TOKEN


    self.Encoder=encoder(self.max_sequence_length,self.d_model,self.hidlayer,self.dropout,self.num_heads,self.nlayers,
                         self.english_to_index,self.START_TOKEN,self.END_TOKEN,self.PADDING_TOKEN
               )
    self.Decoder=decoder(self.max_sequence_length,self.d_model,self.hidlayer,self.dropout,self.num_heads,self.nlayers,
                         self.lang_to_index,self.START_TOKEN,self.END_TOKEN,self.PADDING_TOKEN
               )
    self.linearlayer=nn.Linear(self.d_model,self.vocab_size)

  def forward(self,english_sentences,hindi_sentences,encodermask=None,decodermask=None,decoder_cross_attention_mask=None):
    x=self.Encoder(english_sentences,padding_mask=encodermask,start=False,end=False)
    y=self.Decoder(x,hindi_sentences,padding_mask=decodermask,look_ahead_mask=decoder_cross_attention_mask,start=True,end=True,)
    y=self.linearlayer(y)
    #print('output of linear layer',y.shape)
    #softmaxoutput=torch.log_softmax(y,dim=-1)
    #print('output of softmax layer',softmaxoutput.shape)
    #return softmaxoutput
    #predict next tokens
    #predicted_tokens = torch.argmax(softmaxoutput, dim=-1)  # Shape: (3, 5)
    # predict next token
    #predicted_next_token = torch.argmax(softmaxoutput[-1, -1, :])  # Shape: ()
    # predict next tokens
    #predicted_next_tokens = torch.argmax(softmaxoutput[:, -1, :],dim=-1)  # Shape: ()
    return y #(batch,seqlen,vocabsize)



In [8]:
#vocab

START_TOKEN = "<START>"
END_TOKEN = "<END>"
PADDING_TOKEN = "<PAD>"

english_vocabulary = [' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/',
                      '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', '<', '=', '>', '?', '@',
                      'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L',
                      'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X',
                      'Y', 'Z', '[', '\\', ']', '^', '_', '`',
                      'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l',
                      'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x',
                      'y', 'z', '{', '|', '}', '~', START_TOKEN,PADDING_TOKEN, END_TOKEN]



hindi_vocabulary = [' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/',
                    '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', '<', '=', '>', '?', 'ँ', 'ं', 'ः',
                    'अ', 'आ', 'इ', 'ई', 'उ', 'ऊ', 'ऋ', 'ऌ', 'ए', 'ऐ', 'ओ', 'औ',
                    'क', 'ख', 'ग', 'घ', 'ङ', 'च', 'छ', 'ज', 'झ', 'ञ', 'ट', 'ठ', 'ड', 'ढ', 'ण',
                    'त', 'थ', 'द', 'ध', 'न', 'प', 'फ', 'ब', 'भ', 'म', 'य', 'र', 'ल', 'व', 'श', 'ष', 'स', 'ह',
                    '़', 'ऽ', 'ा', 'ि', 'ी', 'ु', 'ू', 'ृ', 'ॄ', 'ॅ', 'े', 'ै', 'ॉ', 'ो', 'ौ', '्', 'ॐ',
                    '०', '१', '२', '३', '४', '५', '६', '७', '८', '९', START_TOKEN,PADDING_TOKEN, END_TOKEN]

index_to_hindi = {k:v for k,v in enumerate(hindi_vocabulary)}
hindi_to_index = {v:k for k,v in enumerate(hindi_vocabulary)}
index_to_english = {k:v for k,v in enumerate(english_vocabulary)}
english_to_index = {v:k for k,v in enumerate(english_vocabulary)}

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
#dataset creation

data=pd.read_csv('/content/drive/MyDrive/ADeepLearning/hindi_english_parallel2.csv')
data.head()

,English,Hindi
0,Help!,बचाओ!
1,Jump.,उछलो.
2,Jump.,कूदो.
3,Jump.,छलांग.
4,Hello!,नमस्ते।


In [11]:
data.dropna(inplace=True)
data.shape

(130162, 2)

In [12]:
data.isnull().sum()

,0
English,0
Hindi,0


In [13]:

sample=data[:10000].copy()
engsen1=sample['English'].to_list()
hindisen1=sample['Hindi'].to_list()
# Ensure all values are strings
hindisen1 = ["" if isinstance(sent, float) else str(sent) for sent in hindisen1]
engsen1 = ["" if isinstance(sent, float) else str(sent) for sent in engsen1]


In [14]:
# Find new unique characters
unique_chars_in_data = set("".join(hindisen1))
missing_chars = unique_chars_in_data - set(hindi_vocabulary )

# Add missing characters
hindi_vocabulary .extend(missing_chars)
hindi_vocabulary  = sorted(set(hindi_vocabulary ))  # Ensure uniqueness and order

print("✅ Updated Hindi Vocabulary Size:", len(hindi_vocabulary ))
print("✅ New Characters Added:", missing_chars)

✅ Updated Hindi Vocabulary Size: 206
✅ New Characters Added: {'[', 'P', 'J', '\u200c', 'v', 'p', 'b', 'l', '|', 'Z', 'ل', 'K', 'e', 'g', '`', 'L', '\u200d', '̈', 'a', 'Q', 'd', 'ॊ', '”', 'ऒ', 'G', '☺', 'س', '\u200b', 'c', 'ح', 'ळ', 'z', 'q', 'ॡ', '।', 'r', '“', 'y', 'H', '\\', '_', 'N', 'T', 'W', 'm', 'م', 'B', '°', 'j', ';', 'M', '·', 'ड़', 'ज़', '॰', 'ख़', 'n', 'ऎ', 'ढ़', '♫', 'x', 'ا', 'ج', 'D', 'E', 'ऍ', '~', 'w', 'X', 'o', 's', 'h', ']', '{', '☻', 'i', 't', '\u200e', 'Y', '‘', 'u', 'F', 'C', '@', 'U', 'ऑ', 'S', 'A', 'ٕ', 'f', 'k', 'I', 'O', 'V', '́', 'R', 'ت'}


In [15]:


index_to_hindi = {k:v for k,v in enumerate(hindi_vocabulary)}
hindi_to_index = {v:k for k,v in enumerate(hindi_vocabulary)}
index_to_english = {k:v for k,v in enumerate(english_vocabulary)}
english_to_index = {v:k for k,v in enumerate(english_vocabulary)}

In [16]:
len(index_to_hindi)

206

In [17]:
max_sequence_length = 5000  # Maximum allowed characters
min_sequence_length = 1  # Minimum required characters
# Ensure all values are strings
hindisen1 = ["" if isinstance(sent, float) else str(sent) for sent in hindisen1]
engsen1 = ["" if isinstance(sent, float) else str(sent) for sent in engsen1]

# Function to check if a sentence contains only valid tokens
def is_valid_tokens(sentence, vocab):
    for token in list(set(sentence)):  # Ensure unique characters are checked
        if token not in vocab:
            return False
    return True

# Function to check if sentence length is within min & max limits
def is_valid_length(sentence, min_length, max_length):
    sentence_length = len(sentence)  # No need to convert to list explicitly
    return min_length <= sentence_length < max_length  # Ensuring it fits the range

valid_sentence_indices = []
for index in range(len(hindisen1)):
    hindi_sentence, english_sentence = hindisen1[index], engsen1[index]

    if (is_valid_length(hindi_sentence, min_sequence_length, max_sequence_length) and
        is_valid_length(english_sentence, min_sequence_length, max_sequence_length) and
        is_valid_tokens(hindi_sentence, hindi_vocabulary)):

        valid_sentence_indices.append(index)

print(f"Total sentences: {len(hindisen1)}")
print(f"Valid sentences: {len(valid_sentence_indices)}")


Total sentences: 10000
Valid sentences: 10000


In [18]:
hindisen2 = [hindisen1[i] for i in valid_sentence_indices]
engsen2 = [engsen1[i] for i in valid_sentence_indices]


In [19]:
#data creation

In [20]:
batch_size = 30
dataset = TextDataset(engsen2, hindisen2)
train_loader = DataLoader(dataset, batch_size,shuffle=True)
iterator = iter(train_loader)
d_model = 64
ffn_hidden = 2048
num_heads = 8
drop_prob = 0.1
num_layers = 1
max_sequence_length = 6
hn_vocab_size=len(hindi_to_index)

transformer=mytransformer(d_model,ffn_hidden,drop_prob,num_heads,num_layers,
                          hn_vocab_size,english_to_index,hindi_to_index,max_sequence_length,START_TOKEN,END_TOKEN,PADDING_TOKEN)


transformer.Decoder.decoderembeddings.batch_tokenize(['मेरे'],start=True,end=True)

tensor([[ 31, 151, 171, 153, 171,  29]])

In [21]:
#train
batch_size = 30
dataset = TextDataset(engsen2, hindisen2)
train_loader = DataLoader(dataset, batch_size)
iterator = iter(train_loader)
d_model = 64
ffn_hidden = 2048
num_heads = 8
drop_prob = 0.1
num_layers = 8
max_sequence_length = 100
hn_vocab_size=len(hindi_to_index)

transformer=mytransformer(d_model,ffn_hidden,drop_prob,num_heads,num_layers,
                          hn_vocab_size,english_to_index,hindi_to_index,max_sequence_length,START_TOKEN,END_TOKEN,PADDING_TOKEN)



To summarize:

Encoder: Padding mask is applied.

Decoder (Self-Attention): Both padding mask and look-ahead mask are applied.

Decoder (Cross-Attention): Only the padding mask is applied

In [22]:
def create_mask(input, pad):
    # Padding mask: Shape (batch_size, 1, 1, seq_len), where True indicates padding
    pmask = (input == pad).unsqueeze(1).unsqueeze(2)

    # Look-ahead mask: Shape (1, 1, seq_len, seq_len)
    seq_len = input.size(1)  # Get sequence length
    clmask = torch.tril(torch.ones((seq_len, seq_len), dtype=torch.bool, device=input.device))
    clmask = clmask.unsqueeze(0).unsqueeze(0)  # Shape (1, 1, seq_len, seq_len)

    return pmask, clmask

In [23]:
NEG_INFTY = -1e9
max_sequence_length=6
def create_masks(eng_batch, kn_batch):
    num_sentences = len(eng_batch)
    look_ahead_mask = torch.full([max_sequence_length, max_sequence_length] , True)
    look_ahead_mask = torch.triu(look_ahead_mask, diagonal=1)
    encoder_padding_mask = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)
    decoder_padding_mask_self_attention = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)
    decoder_padding_mask_cross_attention = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)

    for idx in range(num_sentences):
      eng_sentence_length, kn_sentence_length = len(eng_batch[idx]), len(kn_batch[idx])
      eng_chars_to_padding_mask = np.arange(eng_sentence_length + 1, max_sequence_length)
      kn_chars_to_padding_mask = np.arange(kn_sentence_length + 1, max_sequence_length)
      encoder_padding_mask[idx, :, eng_chars_to_padding_mask] = True
      encoder_padding_mask[idx, eng_chars_to_padding_mask, :] = True
      decoder_padding_mask_self_attention[idx, :, kn_chars_to_padding_mask] = True
      decoder_padding_mask_self_attention[idx, kn_chars_to_padding_mask, :] = True
      decoder_padding_mask_cross_attention[idx, :, eng_chars_to_padding_mask] = True
      decoder_padding_mask_cross_attention[idx, kn_chars_to_padding_mask, :] = True

    encoder_self_attention_mask = torch.where(encoder_padding_mask, NEG_INFTY, 0)
    decoder_self_attention_mask =  torch.where(look_ahead_mask + decoder_padding_mask_self_attention, NEG_INFTY, 0)
    decoder_cross_attention_mask = torch.where(decoder_padding_mask_cross_attention, NEG_INFTY, 0)
    return encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask



# encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask=create_masks(eng_batch, hn_batch)
# encoder_self_attention_mask.shape, decoder_self_attention_mask.shape, decoder_cross_attention_mask.shape

In [24]:

#set loss func and init params

criterian = nn.CrossEntropyLoss(ignore_index=hindi_to_index[PADDING_TOKEN],
                                reduction='none')

# When computing the loss, we are ignoring cases when the label is the padding token
for params in transformer.parameters():
    if params.dim() > 1:
        nn.init.xavier_uniform_(params)

optim = torch.optim.Adam(transformer.parameters(), lr=1e-4)

In [25]:
dataset.english_sentences[200]

'How old are you?'

In [26]:
# for batch_num, batch in enumerate(iterator):
#     if batch_num%500==0:
#         print(f"Iteration {batch_num}")
#         transformer.train()
#         eng_batch, hn_batch = batch  # Get English and Hindi sentences
#         optim.zero_grad()
#         print(f"English: {eng_batch[29]}")
#         print(f"hindi Translation: {hn_batch[29]}")
#         break

In [27]:
transformer.train()

total_loss = 0
num_epochs = 100

for epoch in range(num_epochs):
    print("*"*500)
    print(f"Epoch {epoch+1}")
    print("*"*500)
    iterator = iter(train_loader)

    for batch_num, batch in enumerate(iterator):
        if batch_num%500==0:
            print(f"Iteration {batch_num}")
            transformer.train()
            eng_batch, hn_batch = batch  # Get English and Hindi sentences
            optim.zero_grad()
            print(f"English: {eng_batch[0]}")
            print(f"hindi Translation: {hn_batch[0]}")
            encodermask,clmask=create_mask(transformer.Encoder.encoderembeddings.batch_tokenize(eng_batch,start=False,end=False),0)
            decoderpadmask,crossattentionmask=create_mask(transformer.Decoder.decoderembeddings.batch_tokenize(hn_batch,start=True,end=True),0)
            encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask=create_masks(eng_batch,hn_batch)
            # Get predicted logits
            #predictions = transformer(eng_batch, hn_batch,encoder_self_attention_mask,decoder_self_attention_mask,decoder_cross_attention_mask)
            predictions = transformer(eng_batch, hn_batch,encodermask,decoderpadmask,crossattentionmask)

            #get labels
            targets = transformer.Decoder.decoderembeddings.batch_tokenize(hn_batch,start=True,end=True)
            # Compute loss
            loss = criterian(predictions.view(-1, predictions.shape[-1]), targets.view(-1))
            valid_indicies = (targets.view(-1) != hindi_to_index[PADDING_TOKEN])  # Ignore padding
            loss = loss[valid_indicies].mean()
            loss.backward()
            optim.step()
            total_loss+=loss.item()
            print(f"Loss: {loss.item()}")


            # Generate  predictions
            with torch.no_grad():
                hn_predictions = transformer(eng_batch, hn_batch)
                hn_sentence_predicted = torch.argmax(hn_predictions[0], axis=1)

                # Convert indices to  text
                maxtokens=50
                predicted_sentence = []
                count = 0

                for idx in hn_sentence_predicted:  # Directly iterate over the tensor
                    if idx.item() == hindi_to_index[END_TOKEN]: #or count >= maxtokens:
                      break
                    elif idx.item() == hindi_to_index[START_TOKEN]:
                      continue


                    predicted_sentence.append(index_to_hindi[idx.item()])  # Convert index to Hindi character
                    count += 1  # Increment token count


                predicted_sentence="".join(predicted_sentence)
                print(f"hindi Prediction:{predicted_sentence}")
                print("-"*500)

            #validation text
            transformer.eval()
            count=0

            hn_sentence = [START_TOKEN]
            eng_sentence = ("should we go to the mall?",)
            encodermask,clmask=create_mask(transformer.Encoder.encoderembeddings.batch_tokenize(eng_sentence,start=False,end=False),0)
            decoderpadmask,crossattentionmask=create_mask(transformer.Decoder.decoderembeddings.batch_tokenize(hn_sentence,start=True,end=True),0)

            evalpredictions = transformer(eng_sentence,
                                        hn_sentence,encodermask,decoderpadmask,crossattentionmask)
            evalpredictedsentence=torch.argmax(evalpredictions[0], axis=1)
            eval_predicted_sentence=[]
            for idx in evalpredictedsentence:
                if idx.item() == hindi_to_index[END_TOKEN]: #or count >= maxtokens:
                    break
                elif idx.item() == hindi_to_index[START_TOKEN]:
                    continue
                eval_predicted_sentence.append(index_to_hindi[idx.item()])
                count+=1
            eval_predicted_sentence="".join(eval_predicted_sentence)
            print(f"Evaluation translation (should we go to the mall?) : {eval_predicted_sentence}")
            print("-------------------------------------------")
            #break
    #break

********************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************
Epoch 1
***********************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************

In [28]:
predictions.shape,targets.shape,predictions.view(-1, predictions.shape[-1]).shape, targets.view(-1).shape

(torch.Size([30, 100, 206]),
 torch.Size([30, 100]),
 torch.Size([3000, 206]),
 torch.Size([3000]))

In [29]:
targets.view(-1)[valid_indicies]

tensor([ 31, 149, 132, 163, 125,   1,  29,  31, 115, 133, 154, 175,  14,  29,
         31, 127, 167, 144, 175,  14,  29,  31, 133, 154, 163, 109, 129,  14,
         29,  31, 146, 151, 159, 177, 142, 171, 184,  29,  31, 146, 151, 159,
        177, 127, 163, 153, 184,  29,  31, 156, 163, 160,  13, 156, 163, 160,
          1,  29,  31, 132, 164, 152, 153, 177, 159,   1,  29,  31, 159, 151,
        135, 171,   0, 127, 164,   0, 146, 160, 165, 109,  34,  29,  31, 151,
        172, 109,   0, 138, 165, 127,   0, 160, 167, 108, 184,  29,  31, 149,
        160, 166, 142,   0, 149, 140, 161, 164, 152, 163,   1,  29,  31, 111,
        109, 144, 153,   0, 112,   0, 134, 163, 125, 184,  29,  31, 149, 163,
        160, 153,   0, 146, 164, 127, 154,   0, 134, 163, 125,   1,  29,  31,
        132, 154, 171,   0, 134, 163, 125,   1,  29,  31, 128, 161, 166, 144,
        163,   0, 160, 163, 148, 161, 164, 134, 161, 184,  29,  31, 115, 142,
        177, 142, 151,   1,  29,  31, 159, 160, 165,   1,  29,  

In [30]:
predictions.view(-1, predictions.shape[-1])[:2,:]

tensor([[-1.1522e+00,  7.9413e-01, -1.3460e+00,  9.5054e-02,  1.3747e+00,
          5.6067e-02,  4.4126e-01, -6.0004e-01,  1.0179e+00, -1.0773e-01,
         -6.5318e-01, -4.6085e-01, -3.5201e-01, -4.1922e-01, -1.5108e-01,
         -4.9188e-01, -4.2723e-01, -8.6311e-01, -5.6174e-01, -1.2438e-01,
          8.6003e-01, -5.0773e-01, -1.0953e+00,  6.6582e-01, -1.2090e+00,
         -1.3427e+00,  8.1543e-02,  1.6953e+00,  4.8791e-01,  8.1226e-01,
         -5.1147e-01,  5.7784e+00, -8.1506e-02, -2.9741e-01, -3.7482e-01,
         -4.4172e-01, -1.3505e-01,  1.0631e-02,  1.8369e-01, -5.3866e-01,
         -4.4861e-01,  4.7516e-01,  1.3023e-01,  5.8327e-02, -5.0448e-01,
         -1.4075e+00, -1.2973e+00,  1.0943e-01, -1.0117e+00,  1.5477e+00,
          1.1486e+00, -1.6646e+00, -9.6839e-01, -1.5031e-02, -3.6083e-01,
         -2.0210e+00,  2.4092e-01,  6.9859e-01, -2.2241e-01,  2.9843e-01,
          3.5376e-01, -1.6433e-01, -1.0050e+00, -1.1939e+00,  6.2121e-01,
         -1.0506e+00,  5.0672e-01, -8.

In [31]:
predictions.view(-1, predictions.shape[-1])[:2,:].shape

torch.Size([2, 206])

In [32]:
targets.view(-1, targets.shape[-1])[:2,:]

tensor([[ 31, 149, 132, 163, 125,   1,  29,  30,  30,  30,  30,  30,  30,  30,
          30,  30,  30,  30,  30,  30,  30,  30,  30,  30,  30,  30,  30,  30,
          30,  30,  30,  30,  30,  30,  30,  30,  30,  30,  30,  30,  30,  30,
          30,  30,  30,  30,  30,  30,  30,  30,  30,  30,  30,  30,  30,  30,
          30,  30,  30,  30,  30,  30,  30,  30,  30,  30,  30,  30,  30,  30,
          30,  30,  30,  30,  30,  30,  30,  30,  30,  30,  30,  30,  30,  30,
          30,  30,  30,  30,  30,  30,  30,  30,  30,  30,  30,  30,  30,  30,
          30,  30],
        [ 31, 115, 133, 154, 175,  14,  29,  30,  30,  30,  30,  30,  30,  30,
          30,  30,  30,  30,  30,  30,  30,  30,  30,  30,  30,  30,  30,  30,
          30,  30,  30,  30,  30,  30,  30,  30,  30,  30,  30,  30,  30,  30,
          30,  30,  30,  30,  30,  30,  30,  30,  30,  30,  30,  30,  30,  30,
          30,  30,  30,  30,  30,  30,  30,  30,  30,  30,  30,  30,  30,  30,
          30,  30,  30,  30,  30

In [33]:
targets.view(-1, targets.shape[-1])[:2,:].shape

torch.Size([2, 100])

In [34]:
# Correct reshaping
targets = targets.view(-1)
# Compute loss with correctly shaped targets
loss = criterian(predictions.view(-1, predictions.shape[-1])[:2,:], targets[:2])

# Ignore padding
valid_indices = (targets[:2] != hindi_to_index[PADDING_TOKEN])
loss = loss[valid_indices].mean()
loss

tensor(1.2934, grad_fn=<MeanBackward0>)

In [35]:
targets[:2]

tensor([ 31, 149])

In [36]:
hindi_to_index[PADDING_TOKEN]

30

In [37]:
targets.shape

torch.Size([3000])

In [38]:
# Correct reshaping
targets = targets.view(-1)
# Compute loss with correctly shaped targets
loss = criterian(predictions.view(-1, predictions.shape[-1]), targets)

# Ignore padding
valid_indices = (targets != hindi_to_index[PADDING_TOKEN])
targets.shape,predictions.view(-1, predictions.shape[-1]).shape,loss[valid_indices].shape


(torch.Size([3000]), torch.Size([3000, 206]), torch.Size([363]))

In [39]:
engsent=eng_batch[-1]
hinsent=hn_batch[-1]
print('engsent',engsent)
print('hinsent',hinsent)
hn_predictions = transformer([engsent], [hinsent])
hn_sentence_predicted = torch.argmax(hn_predictions[0], axis=1)

# Convert indices to  text
maxtokens=50
predicted_sentence = []
count = 0

for idx in hn_sentence_predicted:  # Directly iterate over the tensor
    if idx.item() == hindi_to_index[END_TOKEN]:# or count >= maxtokens:
        break
    elif idx.item() == hindi_to_index[START_TOKEN]:
                      continue
    predicted_sentence.append(index_to_hindi[idx.item()])  # Convert index to Hindi character
    count += 1  # Increment token count


predicted_sentence="".join(predicted_sentence)
print(f"hindi Prediction:{predicted_sentence}")

engsent Birds fly.
hinsent पंछी उड़ते हैं।
hindi Prediction:पंछी उड़ते हैं।


In [40]:
engsent

'Birds fly.'

In [41]:
#inference
transformer.eval()
def translate(eng_sentence):
  engsent=[eng_sentence]
  hinsent=[START_TOKEN]
  hn_predictions = transformer(engsent, hinsent)
  hn_sentence_predicted = torch.argmax(hn_predictions[0], axis=1)

  # Convert indices to  text

  maxtokens=100
  predicted_sentence = []
  count = 0

  for idx in hn_sentence_predicted:  # Directly iterate over the tensor
      if idx.item() == hindi_to_index[END_TOKEN] or count >= maxtokens:
          break
      predicted_sentence.append(index_to_hindi[idx.item()])  # Convert index to Hindi character
      count += 1  # Increment token count


  predicted_sentence="".join(predicted_sentence)
  print(f"hindi Prediction:{predicted_sentence}")

In [42]:
translate("I'd like to see her.")

hindi Prediction:<START>ंााष


In [43]:
index_to_hindi[180]

'ज़'